In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import seaborn as sns

import simul_functions as sf


ModuleNotFoundError: No module named 'simul_functions'

In [ ]:
dir= "C:\\Study\\1014\\1014\\NFL_Analysis\\"

In [ ]:
con =sqlite3.connect(dir+"nfl_db.db")

sql_command = """

select c.*,possessionTeam,attacking_team FROM
(
select track.* from
(
    select pf.* from
    (select gameId,playId,event,x,y,s,a,dir,
    lag(x,1) over( PARTITION by gameId,playId,nflId order by time)xl1,
    lag(y,1) over( PARTITION by gameId,playId,nflId order by time)yl1,
    lag(x,2) over( PARTITION by gameId,playId,nflId order by time)xl2,
    lag(y,2) over( PARTITION by gameId,playId,nflId order by time)yl2,
    case when nflId is null then 'Ball' else substr(displayname,1,1)||substr(displayname,instr(displayname,' ')+1 ) end Name,nflId,position,team,playDirection
    from 
    tracking_data)pf 
    where event='pass_forward'
)track
inner JOIN
(
select gameId,playId from tracking_data where event='pass_forward' GROUP by gameId,playId HAVING count(nflid)<=23--11+11+ball
)one_event
on track.gameId=one_event.gameId and track.playId=one_event.playId 
)c
left JOIN
(
select A.*, 
case when possessionTeam=homeTeamAbbr then 'home' else 'away' end attacking_team
from 
(select distinct gameId,playId,possessionTeam from play_data)A
inner JOIN
(select distinct gameid,homeTeamAbbr,visitorTeamAbbr from game_data)B
on A.gameId=B.gameId
)attack
on c.gameId=ATTACk.gameId
and c.playId=attack.playId


"""
td=pd.read_sql_query(sql_command,con)
con.close()

In [ ]:
xp=pd.read_csv('C:\\Study\\1014\\1014\\pass_completion\\xp.csv')

In [ ]:
xp.head()

In [ ]:
searchwords=['thrown away','Thrown away','threw away','throws ball away']
xp['throw_away_pass']=np.where(xp['playDescription'].str.lower().str.findall('|'.join(searchwords)).str.len()>0,1,0 )
xp[xp['throw_away_pass']==1].shape

In [ ]:
xp=xp[xp['throw_away_pass']==0]

In [ ]:
valid_play=xp[['gameId','playId']].drop_duplicates()
xp.shape,valid_play.shape

In [ ]:
ball_position=td[td['Name']=='Ball'][['gameId','playId','x','y','s','a']]
ball_position.head()

In [ ]:
#table required in the following
#attacking team
    # att_tab1: ball and exp player
    # att_tab2: ball and furthers 4 player 
    # att_tab3: ball and closest 2 players 

#Defence team
    # def_tab1: ball and closest 2 players 
    # def_tab2: expected player and closest 4 players

In [ ]:
#Calculating the attributes for expected player and ball

expected_reciever_variables = td[td['team']==td['attacking_team']].merge(xp,how='inner',on=['gameId','playId','nflId'])
expected_reciever_variables=expected_reciever_variables.merge(ball_position,how='inner',on=['gameId','playId'],suffixes=(None,'_ball'))
expected_reciever_variables['d_ball_expR'] = np.sqrt((expected_reciever_variables['x']-expected_reciever_variables['x_ball'])**2+(expected_reciever_variables['y']-expected_reciever_variables['y_ball'])**2)

expected_reciever_variables.shape,expected_reciever_variables[['gameId','playId']].drop_duplicates().shape


In [ ]:
expected_reciever_variables.columns 

In [ ]:
att_tab1 =expected_reciever_variables[['gameId', 'playId', 'x', 'y', 's', 'a', 'dir', 'xl1', 'yl1', 'xl2',
                                                         'yl2','x_ball', 'y_ball', 's_ball', 'a_ball','d_ball_expR']]

new_names = [(i,'xp_'+i) for i in att_tab1.iloc[:, 2:11].columns.values]
att_tab1.rename(columns = dict(new_names), inplace=True)
att_tab1.head()

In [ ]:
att_tab1.xp_xl1.sum()

In [ ]:
reciever_position=td[td['team']==td['attacking_team']].merge(valid_play,how='inner',on=['gameId','playId'])
reciever_position=reciever_position.merge(ball_position,how='inner',on=['gameId','playId'],suffixes=(None,'_ball'))
reciever_position['d_ball_attP'] = np.sqrt((reciever_position['x']-reciever_position['x_ball'])**2+(reciever_position['y']-reciever_position['y_ball'])**2)
reciever_position['inv_order']=reciever_position.groupby(['gameId','playId'])['d_ball_attP'].rank('first',ascending=False).apply(int)
reciever_position['order']=reciever_position.groupby(['gameId','playId'])['d_ball_attP'].rank('first').apply(int)

reciever_position.head()

In [ ]:
att_tab2=reciever_position[reciever_position['inv_order']<=4][['gameId', 'playId', 'x', 'y', 's', 'a', 'dir', 'xl1', 'yl1', 'xl2',
                                                         'yl2','d_ball_attP','inv_order']]

att_tab2['inv_order']=att_tab2['inv_order'].apply(str)

att_tab2= att_tab2.pivot_table(index=['gameId','playId'],columns=['inv_order'],values=['x', 'y', 's', 'a', 'dir', 'xl1', 'yl1', 'xl2',
                                                         'yl2','d_ball_attP'],aggfunc='first')

att_tab2.columns = ['_fur_b2a_'.join(col) for col in att_tab2.columns.values]
att_tab2=att_tab2.reset_index()
att_tab2.head()

In [ ]:
att_tab3=reciever_position[reciever_position['order']<=2][['gameId', 'playId', 'x', 'y', 's', 'a', 'dir', 'xl1', 'yl1', 'xl2',
                                                         'yl2','d_ball_attP','order']]

att_tab3['order']=att_tab3['order'].apply(str)

att_tab3= att_tab3.pivot_table(index=['gameId','playId'],columns=['order'],values=['x', 'y', 's', 'a', 'dir', 'xl1', 'yl1', 'xl2',
                                                         'yl2','d_ball_attP'],aggfunc='first')

att_tab3.columns = ['_clos_b2a_'.join(col) for col in att_tab3.columns.values]
att_tab3=att_tab3.reset_index()
att_tab3.head()

In [ ]:
defence_position=td[(td['team']!=td['attacking_team'])&(td['Name']!='Ball')].merge(valid_play,how='inner',on=['gameId','playId'])
defence_position=defence_position.merge(ball_position,how='inner',on=['gameId','playId'],suffixes=(None,'_ball'))
defence_position['d_ball_defP'] = np.sqrt((defence_position['x']-defence_position['x_ball'])**2+(defence_position['y']-defence_position['y_ball'])**2)
defence_position['order']=defence_position.groupby(['gameId','playId'])['d_ball_defP'].rank('first').apply(int)

defence_position.head()

In [ ]:
def_tab1=defence_position[defence_position['order']<=2][['gameId', 'playId', 'x', 'y', 's', 'a', 'dir', 'xl1', 'yl1', 'xl2',
                                                         'yl2','d_ball_defP','order']]

def_tab1['order']=def_tab1['order'].apply(str)

def_tab1= def_tab1.pivot_table(index=['gameId','playId'],columns=['order'],values=['x', 'y', 's', 'a', 'dir', 'xl1', 'yl1', 'xl2',
                                                         'yl2','d_ball_defP'],aggfunc='first')

def_tab1.columns = ['_clos_b2d_'.join(col) for col in def_tab1.columns.values]
def_tab1=def_tab1.reset_index()
def_tab1.head()

In [ ]:
defence_position=defence_position.merge(expected_reciever_variables[['gameId', 'playId','x', 'y']],how='inner',on=['gameId','playId'],suffixes=(None,'_xp'))
defence_position['d_xp_defP'] = np.sqrt((defence_position['x']-defence_position['x_xp'])**2+(defence_position['y']-defence_position['y_xp'])**2)
defence_position['order_xp']=defence_position.groupby(['gameId','playId'])['d_xp_defP'].rank('first').apply(int)

defence_position.head()

In [ ]:
def_tab2=defence_position[defence_position['order_xp']<=4][['gameId', 'playId', 'x', 'y', 's', 'a', 'dir', 'xl1', 'yl1', 'xl2',
                                                         'yl2','d_xp_defP','order_xp']]

def_tab2['order_xp']=def_tab2['order_xp'].apply(str)

def_tab2= def_tab2.pivot_table(index=['gameId','playId'],columns=['order_xp'],values=['x', 'y', 's', 'a', 'dir', 'xl1', 'yl1', 'xl2',
                                                         'yl2','d_xp_defP'],aggfunc='first')

def_tab2.columns = ['_clos_xp2d_'.join(col) for col in def_tab2.columns.values]
def_tab2=def_tab2.reset_index()
def_tab2.head()

In [ ]:
input_data=att_tab1.merge(att_tab2,how='inner',on=['gameId','playId'])
input_data=input_data.merge(att_tab3,how='inner',on=['gameId','playId'])
input_data=input_data.merge(def_tab1,how='inner',on=['gameId','playId'])
input_data=input_data.merge(def_tab2,how='inner',on=['gameId','playId'])

input_data.columns.values

In [ ]:
pass_events=xp[['gameId','playId','event']]
pass_events.drop_duplicates(inplace=True)
pass_events.dropna(inplace=True)
pass_events['target']=np.where((pass_events['event']=='pass_outcome_caught')|(pass_events['event']=='pass_outcome_touchdown'),1,0)


In [ ]:
input_data=input_data.merge(pass_events[['gameId','playId','target']],how='inner',on=['gameId','playId'])

In [ ]:
expl_var=['xp_x', 'xp_y', 'xp_s', 'xp_a', 'xp_dir',
       'xp_xl1', 'xp_yl1', 'xp_xl2', 'xp_yl2', 'x_ball', 'y_ball',
       's_ball', 'a_ball', 'd_ball_expR', 'a_fur_b2a_1', 'a_fur_b2a_2',
       'a_fur_b2a_3', 'a_fur_b2a_4', 'd_ball_attP_fur_b2a_1',
       'd_ball_attP_fur_b2a_2', 'd_ball_attP_fur_b2a_3',
       'd_ball_attP_fur_b2a_4', 'dir_fur_b2a_1', 'dir_fur_b2a_2',
       'dir_fur_b2a_3', 'dir_fur_b2a_4', 's_fur_b2a_1', 's_fur_b2a_2',
       's_fur_b2a_3', 's_fur_b2a_4', 'x_fur_b2a_1', 'x_fur_b2a_2',
       'x_fur_b2a_3', 'x_fur_b2a_4', 'xl1_fur_b2a_1', 'xl1_fur_b2a_2',
       'xl1_fur_b2a_3', 'xl1_fur_b2a_4', 'xl2_fur_b2a_1', 'xl2_fur_b2a_2',
       'xl2_fur_b2a_3', 'xl2_fur_b2a_4', 'y_fur_b2a_1', 'y_fur_b2a_2',
       'y_fur_b2a_3', 'y_fur_b2a_4', 'yl1_fur_b2a_1', 'yl1_fur_b2a_2',
       'yl1_fur_b2a_3', 'yl1_fur_b2a_4', 'yl2_fur_b2a_1', 'yl2_fur_b2a_2',
       'yl2_fur_b2a_3', 'yl2_fur_b2a_4', 'a_clos_b2a_1', 'a_clos_b2a_2',
       'd_ball_attP_clos_b2a_1', 'd_ball_attP_clos_b2a_2',
       'dir_clos_b2a_1', 'dir_clos_b2a_2', 's_clos_b2a_1', 's_clos_b2a_2',
       'x_clos_b2a_1', 'x_clos_b2a_2', 'xl1_clos_b2a_1', 'xl1_clos_b2a_2',
       'xl2_clos_b2a_1', 'xl2_clos_b2a_2', 'y_clos_b2a_1', 'y_clos_b2a_2',
       'yl1_clos_b2a_1', 'yl1_clos_b2a_2', 'yl2_clos_b2a_1',
       'yl2_clos_b2a_2', 'a_clos_b2d_1', 'a_clos_b2d_2',
       'd_ball_defP_clos_b2d_1', 'd_ball_defP_clos_b2d_2',
       'dir_clos_b2d_1', 'dir_clos_b2d_2', 's_clos_b2d_1', 's_clos_b2d_2',
       'x_clos_b2d_1', 'x_clos_b2d_2', 'xl1_clos_b2d_1', 'xl1_clos_b2d_2',
       'xl2_clos_b2d_1', 'xl2_clos_b2d_2', 'y_clos_b2d_1', 'y_clos_b2d_2',
       'yl1_clos_b2d_1', 'yl1_clos_b2d_2', 'yl2_clos_b2d_1',
       'yl2_clos_b2d_2', 'a_clos_xp2d_1', 'a_clos_xp2d_2',
       'a_clos_xp2d_3', 'a_clos_xp2d_4', 'd_xp_defP_clos_xp2d_1',
       'd_xp_defP_clos_xp2d_2', 'd_xp_defP_clos_xp2d_3',
       'd_xp_defP_clos_xp2d_4', 'dir_clos_xp2d_1', 'dir_clos_xp2d_2',
       'dir_clos_xp2d_3', 'dir_clos_xp2d_4', 's_clos_xp2d_1',
       's_clos_xp2d_2', 's_clos_xp2d_3', 's_clos_xp2d_4', 'x_clos_xp2d_1',
       'x_clos_xp2d_2', 'x_clos_xp2d_3', 'x_clos_xp2d_4',
       'xl1_clos_xp2d_1', 'xl1_clos_xp2d_2', 'xl1_clos_xp2d_3',
       'xl1_clos_xp2d_4', 'xl2_clos_xp2d_1', 'xl2_clos_xp2d_2',
       'xl2_clos_xp2d_3', 'xl2_clos_xp2d_4', 'y_clos_xp2d_1',
       'y_clos_xp2d_2', 'y_clos_xp2d_3', 'y_clos_xp2d_4',
       'yl1_clos_xp2d_1', 'yl1_clos_xp2d_2', 'yl1_clos_xp2d_3',
       'yl1_clos_xp2d_4', 'yl2_clos_xp2d_1', 'yl2_clos_xp2d_2',
       'yl2_clos_xp2d_3', 'yl2_clos_xp2d_4']

In [ ]:
for col in input_data.columns.tolist():
    val_col=[]
    na_prc= input_data[col].isnull().sum()/len(input_data[col])
    if na_prc>0.01: print(col, str(na_prc))

In [ ]:
input_data.shape,input_data.dropna().shape

In [ ]:
input_data.dropna(inplace=True)

In [ ]:
input_data.to_csv("input_data_135.csv",index=False)

In [ ]:
#This is only for an example for prediction in the paper
input_data= input_data[~((input_data.gameId==2018123000)&(input_data.playId==2528))]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

# for modeling
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [ ]:
X = input_data[expl_var]
y = input_data[['target']]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
X_norm=scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=0)

In [ ]:
np.array(X_test).shape

In [ ]:
# build a model
model = Sequential()
model.add(Dense(16, input_shape=(X.shape[1],), activation='relu')) # Add an input shape! (features,)
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary() 

# compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

# early stopping callback
# This callback will stop the training when there is no improvement in  
# the validation loss for 10 consecutive epochs.  
es = EarlyStopping(monitor='val_accuracy', 
                                   mode='max', # don't minimize the accuracy!
                                   patience=10,
                                   restore_best_weights=True)

# now we just update our model fit call
history = model.fit(X_train,
                    y_train,
                    callbacks=[es],
                    epochs=80, # you can set this to a big number!
                    batch_size=10,
                    validation_split=0.2,
                    shuffle=True,
                    verbose=1)

In [ ]:
history_dict = history.history
# Learning curve(Loss)
# let's see the training and validation loss by epoch

# loss
loss_values = history_dict['loss'] # you can change this
val_loss_values = history_dict['val_loss'] # you can also change this

# range of X (no. of epochs)
epochs = range(1, len(loss_values) + 1) 

# plot
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'orange', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Learning curve(accuracy)
# let's see the training and validation accuracy by epoch

# accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# range of X (no. of epochs)
epochs = range(1, len(acc) + 1)

# plot
# "bo" is for "blue dot"
plt.plot(epochs, acc, 'bo', label='Training accuracy')
# orange is for "orange"
plt.plot(epochs, val_acc, 'orange', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# this is the max value - should correspond to
# the HIGHEST train accuracy
np.max(val_acc)

In [ ]:
#n fold validaton for logmodel, svm and random forest

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from sklearn.neural_network import MLPClassifier


In [ ]:
rf=RandomForestClassifier(n_estimators=40)
rf.fit(X_train,y_train)
rf.score(X_test,y_test)

In [ ]:
svm_model = svm.SVC(gamma='scale')
svm_model.fit(X_train, y_train) 
svm_model.score(X_test,y_test)


In [ ]:
%%time
clf = MLPClassifier(hidden_layer_sizes=(100),random_state=1, max_iter=80)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

In [ ]:
import pickle

# save
with open('NN_model.pkl','wb') as f:
    pickle.dump(clf,f)

# load
#with open('model.pkl', 'rb') as f:
#    clf2 = pickle.load(f)
#
#clf2.predict(X[0:1])
#
#

In [ ]:
import pickle
scalerfile = 'NN_scaler.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))


#import pickle
#scalerfile = 'scaler.sav'
#scaler = pickle.load(open(scalerfile, 'rb'))
#test_scaled_set = scaler.transform(test_set)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
folds=10
scores_svm= cross_val_score(svm.SVC(gamma='scale'),X_norm,y,cv=folds)
scores_rf= cross_val_score(RandomForestClassifier(n_estimators=40),X_norm,y,cv=folds)
scores_nn= cross_val_score(clf,X_norm,y,cv=folds)



In [ ]:
print("Accuracy SVM: %0.2f (+/- %0.2f)" % (scores_svm.mean(),scores_svm.std()*2))
print("Accuracy RF : %0.2f (+/- %0.2f)" % (scores_rf.mean(),scores_rf.std()*2))
print("Accuracy NN : %0.2f (+/- %0.2f)" % (scores_nn.mean(),scores_nn.std()*2))

In [ ]:
X = input_data[expl_var]
y = input_data[['target']]
X[expl_var]= scaler.fit_transform(X[expl_var])


In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import roc_auc_score

def calculate_auc(model, X, y, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    auc_scores = []
    accuracy_score=[]
    
    for train_index, test_index in kf.split(X):
        X_train, X_test, y_train, y_test = (X.iloc[train_index], 
                                            X.iloc[test_index], 
                                            y.iloc[train_index], 
                                            y.iloc[test_index])

        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)[:, 1]
        
        auc = roc_auc_score(y_test, y_pred)
        auc_scores.append(auc)
        
        acc=model.score(X_test,y_test)
        accuracy_score.append(acc)
    
    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)
    
    #print("AUC scores:", auc_scores)
    print("Mean AUC:", mean_auc)
    print("Standard deviation of AUC:", std_auc)
    
    #print("Accuracy:", accuracy_score)
    print("Mean Acc:", np.mean(accuracy_score))
    print("Standard deviation of acc:", np.std(accuracy_score))
    return auc_scores


In [ ]:
#n fold validaton for logmodel, svm and random forest

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from sklearn.neural_network import MLPClassifier


In [ ]:
lr=LogisticRegression()
lr_auc=calculate_auc(lr, X, y, n_splits=10) 

In [ ]:
rf=RandomForestClassifier(n_estimators=40)
rf_auc=calculate_auc(rf, X, y, n_splits=10) 


In [ ]:
svm_model = svm.SVC(gamma='scale',probability=True)
svm_auc=calculate_auc(svm_model, X, y, n_splits=10) 

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(100),random_state=1, max_iter=100)
clf_auc=calculate_auc(clf, X, y, n_splits=10) 

In [ ]:
ense_auc=[*lr_auc, *rf_auc,*svm_auc,*clf_auc]

In [ ]:
np.mean(ense_auc),np.std(ense_auc)